# Spark

Apache's framework for distributed processing

- Initialisation Process

- basic dice exemple

---

## guide


nous vle konnen exactement mete en claire concept poun kember yo poun k travay avec spark.

The first thing a Spark program must do is to create a SparkContext object, which tells Spark how to access a cluster. To create a SparkContext you first need to build a SparkConf object that contains information about your application.

The appName parameter is a name for your application to show on the cluster UI. master is a Spark, Mesos or YARN cluster URL, or a special “local” string to run in local mode. In practice, when running on a cluster, you will not want to hardcode master in the program, but rather launch the application with spark-submit and receive it there. However, for local testing and unit tests, you can pass “local” to run Spark in-process.


> Heuresment pou nous msambler jwenn deux methode pou demarrer Spark sou machine la.

Fok nous set driver/host computer a.



key concepts for Spark:

- RDD
- transformer
- action




In [ ]:
from pyspark import SparkContext, SparkConf
master="local[3]"
appName="datamining-spark"
conf = SparkConf().set("spark.driver.host","127.0.0.1").setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)

# That would work: SparkSession.builder.config('spark.driver.host', '127.0.0.1').getOrCreate()


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config('spark.driver.host', '127.0.0.1') \
    .master("local[1]") \
    .appName("Word Count") \
    .getOrCreate()

sparkSc = spark.sparkContext

## Nou pral demarrer exemple Dice la.

In [ ]:
import random as rd 
n = 100
rd.seed(1)
simulations = range(100)



- SC cluster/context

In [ ]:
headsc = (sc.parallelize(simulations)
        .map(lambda _: rd.random())
        .filter(lambda dice: dice<0.2)
        .count()
    )

In [ ]:
print(headsc)
tails = n -headsc
ratio = headsc / n
print(f'number of tails: {tails}', f'ratio: {ratio}')

- sparkSc cluster/context

In [ ]:
hsp = (sparkSc.parallelize(simulations)
        .map(lambda _: rd.random())
        .filter(lambda dice: dice<0.3)
        .count()
    )

print(hsp)
t = n -hsp
rat = hsp / n
print(f'number of tails: {t}', f'ratio: {rat}')


In [ ]:
#sparkSc.stop()
#sc.stop()

## Map VS FlatMap

---

In [ ]:
%%writefile input.txt
Hello world
Another line
Yet another line
The last line

In [ ]:
from pyspark import SparkContext, SparkConf
mast="local[2]"
Name="mapVSflatmap"
con = SparkConf().set("spark.driver.host","127.0.0.1").setAppName(Name).setMaster(mast)
spC = SparkContext(conf=con)

In [ ]:
# we can get access to the inut.txt
spC.textFile("./input.txt")\
    .map(lambda x: x.split()) \
        .collect()

In [ ]:
spC.textFile("./input.txt")\
    .flatMap(lambda x: x.split()) \
        .collect()

In [ ]:
#spC.stop()

## PairRDD

---

In [7]:
from pyspark import SparkContext, SparkConf
Mastery="local"
NameApp="PairRDD"
conn = SparkConf().set("spark.driver.host","127.0.0.1").setAppName(NameApp).setMaster(Mastery)
sconn = SparkContext(conf=conn)

In [11]:
%%writefile sales.txt
#ID DATE        STORE  PRODUCT AMOUNT
100 12/13/2001  34     #345   375.43
103 12/13/2004  31     #325   375.90
 10 12/13/2011   3     #145    23.00

Overwriting sales.txt


In [12]:
sconn.textFile("./sales.txt")\
    .take(2)

['#ID DATE        STORE  PRODUCT AMOUNT',
 '100 12/13/2001  34     #345   375.43']

In [13]:
sconn.textFile("./sales.txt")\
    .top(2)

['103 12/13/2004  31     #325   375.90',
 '100 12/13/2001  34     #345   375.43']

In [20]:
sconn.textFile("./sales.txt")\
    .map(lambda x: x.split()) \
    .take(2)

[['#ID', 'DATE', 'STORE', 'PRODUCT', 'AMOUNT'],
 ['100', '12/13/2001', '34', '#345', '375.43']]

In [28]:
sconn.textFile("./sales.txt")\
    .map(lambda x: x.split()) \
        .filter(lambda x: not x[0].startswith('#'))\
    .take(2)

[['100', '12/13/2001', '34', '#345', '375.43'],
 ['103', '12/13/2004', '31', '#325', '375.90']]

In [31]:
sconn.textFile("./sales.txt")\
    .flatMap(lambda x: x.split()) \
        .filter(lambda x: not x.startswith('#'))\
    .collect()

['DATE',
 'STORE',
 'PRODUCT',
 'AMOUNT',
 '100',
 '12/13/2001',
 '34',
 '375.43',
 '103',
 '12/13/2004',
 '31',
 '375.90',
 '10',
 '12/13/2011',
 '3',
 '23.00']

In [33]:
sconn.textFile("./sales.txt")\
    .map(lambda x: x.split()) \
        .filter(lambda x: not x[0].startswith('#'))\
        .map(lambda x: x[-1])\
    .collect()

['375.43', '375.90', '23.00']

In [35]:
sconn.textFile("./sales.txt")\
    .map(lambda x: x.split()) \
        .filter(lambda x: not x[0].startswith('#'))\
        .map(lambda x: float(x[-1]))\
    .sum()

774.3299999999999

In [36]:
#sconn.stop()